In [379]:
import tensorflow as tf
import numpy as np
import re
import requests

### Getting the text from the name_gen file

In [381]:
r = requests.get("https://cs.stanford.edu/people/karpathy/char-rnn/warpeace_input.txt")
text = r.text

### Proprocessing the text file

In [382]:
### Removing unwanted charaters
# text = text.replace('\n',"")
# text = re.sub("[-.0-9:]","",text)
# text = text.lower()
vocab = sorted(set(text))
vocab_size = len(vocab)
print(vocab_size)

87


### Converting the characters to numbers

In [383]:
char_2_idx = {u:i for i,u in enumerate(vocab)}
idz_2_chat = {i:u for i,u in enumerate(vocab)}

In [403]:
seq_length = 10
x= []
y=[]
for i in range(0,len(text)-seq_length,1):
    inputs = text[i:i+seq_length]
    outputs = text[i+seq_length]
    x_s = [char_2_idx[j] for j in inputs]
    y_s = [char_2_idx[outputs]]
    x.append(x_s)
    y.append(y_s)

In [404]:
y = tf.keras.utils.to_categorical(y)

In [405]:
# x = np.array(x)
# y = np.array(y)

In [406]:
batch = len(x)
x = np.reshape(x,(batch,seq_length,1))


In [407]:
x.shape,y.shape

((3258236, 10, 1), (3258236, 86))

### Building the model



In [422]:
## Building the model
check_dir = 'save_check.hdf5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(check_dir,monitor='loss',save_best_only=True,verbose=1)
modelCallbacks = checkpoint
# batch_size=32

model = tf.keras.Sequential([
#     tf.keras.layers.Embedding(vocab_size,input_shape=(1,seq_length),output_dim=(5)),
    tf.keras.layers.LSTM(units=800,return_sequences=True,input_shape=(seq_length,1)),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units=800,return_sequences=True),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.LSTM(units=800),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(units=y.shape[1],activation='softmax')
])

model.summary()

Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_118 (LSTM)             (None, 10, 800)           2566400   
                                                                 
 dropout_18 (Dropout)        (None, 10, 800)           0         
                                                                 
 lstm_119 (LSTM)             (None, 10, 800)           5123200   
                                                                 
 dropout_19 (Dropout)        (None, 10, 800)           0         
                                                                 
 lstm_120 (LSTM)             (None, 800)               5123200   
                                                                 
 dropout_20 (Dropout)        (None, 800)               0         
                                                                 
 dense_31 (Dense)            (None, 86)              

In [423]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [424]:
epoch_number = 0
filename = 'prediction.txt'
file = open(filename,'w')
file.truncate()
file.close()

class CustomCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs=None):
        global epoch_number
        epoch_number = epoch_number + 1
        filename = 'predictions.txt'
        file = open(filename,'a')
        seed = "looking for"
        pattern = []
        for i in seed:
            pattern.append(char_2_idx[i])
        file.seek(0)
        file.write(f"\n\n Epochs number : {epoch_number}")
        for i in range(100):
            X = np.reshape(pattern,(1, len(pattern) , 1))
            X = X/float(n_vocab) 
            int_prediction = Model.predict(X ,verbose = 0)
            index = tf.random.categorical(predicted,num_samples=1).numpy()[0,-1]
            prediction = idz_2_chat[index]
            file.write(prediction)
            pattern.append(index)
            pattern = pattern[1:len(pattern)]
        file.close()


In [425]:
model.fit(x,y,epochs=10,verbose=1,batch_size=300,callbacks=[CustomCallbacks(),modelCallbacks])

Epoch 1/10
  100/10861 [..............................] - ETA: 4:08:17 - loss: 3.1078

KeyboardInterrupt: 

In [418]:
from tqdm import tqdm
seed="sandy"
test_txt = []
prediction =[]
for i in tqdm(range(1,101)):
    j = 1
    prediction = [char_2_idx[j] for j in seed]
    prediction = np.array(prediction)
    prediction = np.expand_dims([prediction],axis=-1)
    predicted  = model.predict(prediction,verbose=0)
#     predicted = tf.squeeze(predicted,axis=0)
    predicted = tf.random.categorical(predicted,num_samples=1).numpy()[0,-1]
    text = idz_2_chat[predicted]
    test_txt.append(text)
    seed = seed+text
    seed = seed[j:]
    j = j+1

100%|█████████████████████████████████████████| 100/100 [00:07<00:00, 12.83it/s]


In [419]:
test_txt = ''.join(test_txt)

In [420]:
test_txt

'bxLHYr7OR,x©EBNPm»L\xa0r»qumYCj!82GzXsDlaQHH»R0NHn1tPi7A©Eey6"aG:7Bw/:DJ-cCvHGljG\'gV:\')yX:oyzL.lQS(l.j*'

In [89]:
prediction.shape

(1, 1, 4)

In [97]:
char_2_idx

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [153]:
seed = 'peetevyjev'

In [154]:
seed[5:]

'vyjev'

## Training Continuation

In [ ]:
try:
    model.load_weights(check_dir)
except Exception as error:
    print(f'Error loading in model : {error}')

model.fit(x, y , batch_size = 200, epochs = 10 ,callbacks = [CustomCallback() ,model_callbacks])